# Test Agent A2A Walkthrough

This notebook sends a JSON-RPC message to the Test Agent A2A endpoint and prints the response.

Set `APP_HOST` to your app host (without scheme) or to a full URL.

In [1]:
from __future__ import annotations
import json

from uuid import uuid4

import httpx
from databricks.sdk import WorkspaceClient

APP_HOST = "localhost:8000"

if APP_HOST.startswith("http://") or APP_HOST.startswith("https://"):
    BASE_URL = APP_HOST.rstrip("/")
else:
    BASE_URL = f"http://{APP_HOST}".rstrip("/")

if "localhost" in APP_HOST:
    headers: dict[str, str] = {}
else:
    w = WorkspaceClient()
    headers = w.config.authenticate()

A2A_TEST_URL = f"{BASE_URL}/test-agent/"
A2A_GATEWAY_URL = f"{BASE_URL}/a2a"
print("A2A_TEST_URL:", A2A_TEST_URL)
print("A2A_GATEWAY_URL:", A2A_GATEWAY_URL)

A2A_TEST_URL: http://localhost:8000/test-agent/
A2A_GATEWAY_URL: http://localhost:8000/a2a


In [2]:
def send_a2a_message(url: str, text: str) -> dict:
    payload = {
        "jsonrpc": "2.0",
        "id": uuid4().hex,
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [{"kind": "text", "text": text}],
                "message_id": uuid4().hex,
            }
        },
    }
    resp = httpx.post(url, json=payload, headers=headers, timeout=60.0)
    resp.raise_for_status()
    return resp.json()


def extract_messages(a2a_response: dict) -> list[dict]:
    result = a2a_response.get("result", {})
    messages = []
    for msg in result.get("history", []) or []:
        parts = [
            part.get("text")
            for part in msg.get("parts", [])
            if part.get("kind") == "text"
        ]
        messages.append(
            {
                "role": msg.get("role", "unknown"),
                "message_id": msg.get("messageId") or msg.get("message_id"),
                "parts": parts,
            }
        )
    for artifact in result.get("artifacts", []) or []:
        parts = [
            part.get("text")
            for part in artifact.get("parts", [])
            if part.get("kind") == "text"
        ]
        if parts:
            messages.append(
                {
                    "role": "agent",
                    "message_id": artifact.get("artifactId"),
                    "parts": parts,
                }
            )
    return messages

In [3]:
response = send_a2a_message(A2A_TEST_URL, "hello from notebook")
print("TEST AGENT RAW:")
print(response)
print("TEST AGENT MESSAGES:")
print(extract_messages(response))

# Invoke a Databricks agent endpoint via the registry A2A gateway.
mas_payload = {"agent_id": "mas-45a56b70-endpoint", "input": "hello from A2A"}
response = send_a2a_message(A2A_GATEWAY_URL, json.dumps(mas_payload))
print("MAS RAW:")
print(response)
print("MAS MESSAGES:")
print(extract_messages(response))

TEST AGENT RAW:
{'id': '8d214fcd317843f1be8fe2b9533e6812', 'jsonrpc': '2.0', 'result': {'artifacts': [{'artifactId': 'babcbe7f-c589-4fcf-a09d-187ee87de04d', 'name': 'test_agent_response', 'parts': [{'kind': 'text', 'text': '{\n  "message": "hello world from Test Agent",\n  "handshake": {\n    "context_id": "033799de-5beb-457c-9360-a993a3215cc1",\n    "task_id": "1b96f6d7-66ce-48d4-add5-b17326120942",\n    "message_id": "5eca2cacbb6242f7b5d36333bb853185"\n  },\n  "request": {\n    "input_text": "hello from notebook"\n  },\n  "history_size": 1\n}'}]}], 'contextId': '033799de-5beb-457c-9360-a993a3215cc1', 'history': [{'contextId': '033799de-5beb-457c-9360-a993a3215cc1', 'kind': 'message', 'messageId': '5eca2cacbb6242f7b5d36333bb853185', 'parts': [{'kind': 'text', 'text': 'hello from notebook'}], 'role': 'user', 'taskId': '1b96f6d7-66ce-48d4-add5-b17326120942'}], 'id': '1b96f6d7-66ce-48d4-add5-b17326120942', 'kind': 'task', 'status': {'state': 'completed', 'timestamp': '2026-02-09T18:49:21

In [19]:
import pprint
clean_message = json.loads(extract_messages(response)[-1]['parts'][0])['result']['payload']['output'][0]['content'][0]['text']
pprint.pprint(clean_message)


("Hello! Welcome! I'm here to help you with questions about the Bixi bike "
 'share network. \n'
 '\n'
 'I can assist you with queries about:\n'
 '- Station information and locations\n'
 '- Bike availability and usage patterns\n'
 '- Trip data and statistics\n'
 '- Network performance metrics\n'
 '- And other Bixi-related data\n'
 '\n'
 'What would you like to know about the Bixi bike share system?')
